In [ ]:
!pip install git+https://github.com/Navu45/image-merging.git

In [ ]:
import torch
from diffusers import DDIMScheduler, DDIMInverseScheduler
from diffusers.utils import load_image, make_image_grid

from models.pix2pix_image import StableDiffusionPix2PixImagePipeline


In [ ]:
pipeline = StableDiffusionPix2PixImagePipeline.from_pretrained(
    'lambdalabs/sd-image-variations-diffusers', revision="v2.0",
    torch_dtype=torch.float16,
    safety_checker=None,
)
pipeline.scheduler = DDIMScheduler.from_config(pipeline.scheduler.config)
pipeline.inverse_scheduler = DDIMInverseScheduler.from_config(pipeline.scheduler.config)
pipeline.enable_model_cpu_offload()

In [ ]:
cat_painting = load_image("https://github.com/pix2pixzero/pix2pix-zero/raw/main/assets/test_images/cats/cat_6.png").convert("RGB").resize((512, 512))
dog_image = load_image('https://images.unsplash.com/photo-1518020382113-a7e8fc38eac9?ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D&auto=format&fit=crop&w=2517&q=80').convert("RGB").resize((512, 512))
make_image_grid([cat_painting, dog_image], 1, 2)

In [ ]:
cat_painting_embeds = pipeline.encode_image(cat_painting)
dog_image_embeds = pipeline.encode_image(dog_image)

In [ ]:
generator = torch.manual_seed(0)
inv_latents, inv_image = pipeline.invert(source_image=cat_painting, prompt_embeds=cat_painting_embeds,
                              generator=generator, guidance_scale=3.0, return_dict=False)

In [ ]:
image = pipeline.vae.decode(inv_latents / pipeline.vae.config.scaling_factor, return_dict=False)[0]
inv_image[0]

In [ ]:
image = pipeline(
    source_embeds=cat_painting_embeds,
    target_embeds=dog_image_embeds,
    num_inference_steps=50,
    cross_attention_guidance_amount=0.15,
    # guidance_scale=3.0,
    generator=generator,
    latents=inv_latents,
).images[0]
image